In [ ]:
import pandas as pd

df=pd.read_csv('/content/drive/MyDrive/PFE/Data sets/fraudTest.csv')
df=pd.DataFrame(df)

In [ ]:
df.columns

In [ ]:
len(df)

In [ ]:
df.columns

In [ ]:
df.drop(columns=['first','last','trans_num','Unnamed: 0','city_pop','unix_time'],inplace=True)


In [ ]:
plt.scatter(df.loc[df['is_fraud'] == 0]['trans_date_trans_time'], df.loc[df['is_fraud'] == 0]['amt'], label="Class #0", alpha=0.5, linewidth=0.15)
plt.scatter(df.loc[df['is_fraud'] == 1]['trans_date_trans_time'], df.loc[df['is_fraud'] == 1]['amt'], label="Class #1", alpha=0.5, linewidth=0.15,c='r')
plt.show()

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes


In [ ]:
#La conversion du code postal en une variable catégorielle facilite le regroupement des transactions par zone géographique,permettant ainsi une meilleure interprétation des schémas de fraude spécifiques à certaines régions ou quartiers.
df['cc_num'] = pd.Categorical(df['cc_num'])
df['zip'] = pd.Categorical(df['zip'])


In [ ]:
# Assurez-vous que la colonne de date est de type datetime
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])
df['dob'] = pd.to_datetime(df['dob'],format='%Y-%m-%d')
df['age_cust'] = (df['trans_date_trans_time'].dt.year - df['dob'].dt.year).astype(int)#attentionnnnnnnnn
df.drop(columns='dob',inplace=True)


df['age_cust'].head()

In [ ]:
import numpy as np


age_ranges = {
    "Child": (0, 13),
    "Teen": (13, 20),
    "Adult": (20, 65),
    "Senior": (65, None)}
df["age_group"] = np.where(df["age_cust"] < 13, "Child", np.where(df["age_cust"] < 20, "Teen", np.where(df["age_cust"] < 55, "Adult", "Senior")))
df["age_group"] = pd.cut(df["age_cust"], bins=[0, 13, 20, 55, 200], labels=["Child", "Teen", "Adult", "Senior"], include_lowest=True)



fraud_counts_age = df.groupby("age_group")["is_fraud"].sum()
fraud_counts_age

In [ ]:
df['hour'] = df['trans_date_trans_time'].dt.hour
df['day'] = df['trans_date_trans_time'].dt.day_name()
df['month'] = df['trans_date_trans_time'].dt.month

df.head()

In [ ]:
df['state'].unique()

In [ ]:
# Sélectionner les intervalles de confiance pour un certain client (cc_num)
cc_num = 4613314721966
df_selected = df.loc[df['cc_num'] == cc_num]

In [ ]:
df['hourEnc'] = 0
df.loc[df.hour < 5,'hourEnc'] = 1
df.loc[df.hour > 21,'hourEnc'] = 1


In [ ]:
import pandas as pd

# Définir l'index comme étant une indexation temporelle basée sur la colonne 'trans_date_trans_time'
df.index = pd.DatetimeIndex(df["trans_date_trans_time"])

# Trier l'index
df = df.sort_index()


In [ ]:
df['is_weekend'] = (df['trans_date_trans_time'].dt.dayofweek >= 5).astype(int)


In [ ]:
# Créez une nouvelle colonne pour représenter le jour de la semaine
df['day_of_week'] = df['trans_date_trans_time'].dt.dayofweek

# Créez une nouvelle colonne pour représenter les weekends (1 pour weekend, 0 sinon)
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Créez une colonne 'year_week' qui combine l'année et le numéro de semaine
df['year_week'] = df['trans_date_trans_time'].dt.to_period('W').dt.to_timestamp()

# Calculez le nombre de transactions pour chaque 'year_week', 'cc_num', et 'is_weekend'
weekly_transactions = df.groupby(['year_week', 'cc_num', 'is_weekend'])['trans_date_trans_time'].count().reset_index(name='num_transactions')

# Fusionnez les transactions hebdomadaires avec df_train en utilisant 'year_week', 'cc_num', et 'is_weekend' comme clés
df = pd.merge(df, weekly_transactions, on=['year_week', 'cc_num', 'is_weekend'], how='left')






In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class_counts = df["is_fraud"].value_counts()

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))



# Bar Graph
axes[0].bar(class_counts.index, class_counts.values, color=['green', 'red'])
axes[0].set_title('Credit Card Fraud Class Distribution (Bar)')
axes[0].set_xlabel('Class')
axes[0].set_ylabel('Count')
axes[0].set_xticks([0, 1])  # Assurez-vous que l'axe des x a des marques pour 0 et 1
axes[0].set_xticklabels(['0', '1'])  # Étiquettes explicites pour les classes

# Pie Chart

axes[1].pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', colors=['green', 'red'])
axes[1].set_title('Credit Card Fraud Class Distribution (Pie)')

plt.tight_layout()
plt.show()

In [ ]:
df.columns

***last days(rolling)***

In [ ]:
# Pour le dataset d'entraînement
df.index = pd.DatetimeIndex(df["trans_date_trans_time"])
df = df.sort_index()
# Calcul de la moyenne glissante des montants de transaction pour chaque 'cc_num' avec shift
df["avg_amt_1_lastDay"] = df.groupby("cc_num")["amt"].transform(lambda x: x.rolling("1d").mean())
df["avg_amt_7_lastDay"] = df.groupby("cc_num")["amt"].transform(lambda x: x.rolling("7d").mean())
df["avg_amt_30_lastDay"] = df.groupby("cc_num")["amt"].transform(lambda x: x.rolling("30d").mean())



In [ ]:
# Calculating the total amount of transactions excluding the current day
df["total_amt_1_lastDay"] = df.groupby("cc_num")["amt"].transform(lambda x: x.rolling("1d").sum())
df["total_amt_7_lastDay"] = df.groupby("cc_num")["amt"].transform(lambda x: x.rolling("7d").sum())
df["total_amt_30_lastDay"] = df.groupby("cc_num")["amt"].transform(lambda x: x.rolling("30d").sum())



In [ ]:
df.isnull().sum().head(35)

In [ ]:
df

In [ ]:

df_selected = df[(df['cc_num'] == 60416207185	)]

# Affichage des lignes sélectionnées
df_selected



In [ ]:
df['trans_date_trans_time'] = pd.to_datetime(df['trans_date_trans_time'])


***chaque jour***

In [ ]:
# Calculer le total des montants de transaction pour chaque groupe 'cc_num' chauqe mois , semaine , 1 jour
df['total_amt_every_day'] = df.groupby(['cc_num', df['trans_date_trans_time'].dt.date])['amt'].transform('sum')
df['total_amt_per_30_days'] = df.groupby(['cc_num', df['trans_date_trans_time'].dt.month])['amt'].transform('sum')
df['total_amt_7_days'] = df.groupby(['cc_num', pd.Grouper(key='trans_date_trans_time', freq='W')])['amt'].transform('sum')


In [ ]:
# Calculer la moyenne des montants de transaction pour chaque groupe 'cc_num' chauqe mois , semaine , 1 jour
df['avg_amt_every_day'] = df.groupby(['cc_num', df['trans_date_trans_time'].dt.date])['amt'].transform('mean')
df['avg_amt_per_30_days'] = df.groupby(['cc_num', df['trans_date_trans_time'].dt.month])['amt'].transform('mean')
df['avg_amt_per_7_days'] = df.groupby(['cc_num', pd.Grouper(key='trans_date_trans_time', freq='W')])['amt'].transform('mean')



In [ ]:

df_selected = df[(df['cc_num'] == 60416207185	)]

# Affichage des lignes sélectionnées
df_selected



In [ ]:
# Calculer le nombre de transactions par client pour chaque période(1jour,7jours,30jours)
df['total_num_transactions_per_day']= df.groupby(['cc_num', pd.Grouper(key='trans_date_trans_time', freq='D')]).transform('size')
df['total_num_transactions_per_7_days']= df.groupby(['cc_num', pd.Grouper(key='trans_date_trans_time', freq='7D')]).transform('size')
df['total_num_transactions_per_30_days'] = df.groupby(['cc_num', pd.Grouper(key='trans_date_trans_time', freq='30D')]).transform('size')



In [ ]:
# Calculating the number of transactions excluding the current day
df['total_num_transactions_lastday'] = df.groupby("cc_num")["trans_date_trans_time"].transform(lambda x: x.rolling("1d").count())
df['total_num_transactions_last7_days'] = df.groupby("cc_num")["trans_date_trans_time"].transform(lambda x: x.rolling("7d").count())
df['total_num_transactions_last30_days'] = df.groupby("cc_num")["trans_date_trans_time"].transform(lambda x: x.rolling("30d").count())



In [ ]:
# Calculate the number of transactions for each merchant for each client over the last day, 7 days, and 30 days
# Exclude the current day's data by using shift(1)
df['total_num_trans_merchant_client_lastday'] = df.groupby(["cc_num", 'merchant'])["trans_date_trans_time"].transform(lambda x: x.rolling("1d").count())
df['total_num_trans_merchant_client_last7_days'] = df.groupby(["cc_num", 'merchant'])["trans_date_trans_time"].transform(lambda x: x.rolling("7d").count())
df['total_num_trans_merchant_client_last30_days'] = df.groupby(["cc_num", 'merchant'])["trans_date_trans_time"].transform(lambda x: x.rolling("30d").count())




In [ ]:
# Calculer le nombre de transactions par client et marchant pour chaque période(1jour,7jours,30jours)
df['total_num_trans_merchant_per_day']= df.groupby(['cc_num','merchant', pd.Grouper(key='trans_date_trans_time', freq='D')]).transform('size')
df['total_num_trans_merchant_per_7_days']= df.groupby(['cc_num', 'merchant',pd.Grouper(key='trans_date_trans_time', freq='7D')]).transform('size')
df['total_num_trans_merchant_per_30_days'] = df.groupby(['cc_num','merchant', pd.Grouper(key='trans_date_trans_time', freq='30D')]).transform('size')


In [ ]:
df

In [ ]:
# Calculer le total des montants de transaction client et marchant pour chaque groupe 'cc_num' chaque mois , semaine , 1 jour
df['total_amt_client_merchant_every_day'] = df.groupby(['cc_num','merchant', df['trans_date_trans_time'].dt.date])['amt'].transform('sum')
df['total_amt_client_merchant_per_30_days'] = df.groupby(['cc_num','merchant', df['trans_date_trans_time'].dt.month])['amt'].transform('sum')
df['total_amt_client_merchant_7_days'] = df.groupby(['cc_num', 'merchant',pd.Grouper(key='trans_date_trans_time', freq='W')])['amt'].transform('sum')



In [ ]:
df

In [ ]:
# Calculate rolling sums excluding the current day's transactions
df["total_amt_client_merchant_1_lastDay"] = df.groupby(["cc_num",'merchant'])["amt"].transform(lambda x: x.rolling("1d").sum())
df["total_amt_client_merchant_7_lastDay"] = df.groupby(["cc_num",'merchant'])["amt"].transform(lambda x: x.rolling("7d").sum())
df["total_amt_client_merchant_30_lastDay"] = df.groupby(["cc_num",'merchant'])["amt"].transform(lambda x: x.rolling("30d").sum())


In [ ]:
df

In [ ]:
# Calculer la moyenne des montants de transaction pour chaque groupe 'cc_num' chauqe mois , semaine , 1 jour
df['avg_amt_client_merchant_every_day'] = df.groupby(['cc_num','merchant',df['trans_date_trans_time'].dt.date])['amt'].transform('mean')
df['avg_amt_client_merchant_per_30_days'] = df.groupby(['cc_num','merchant', df['trans_date_trans_time'].dt.month])['amt'].transform('mean')
df['avg_amt_client_merchant_per_7_days'] = df.groupby(['cc_num','merchant', pd.Grouper(key='trans_date_trans_time', freq='W')])['amt'].transform('mean')




In [ ]:
len(df.columns)

In [ ]:
df.isnull().sum().tail(60,)

In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    # Convertir les degrés en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Rayon moyen de la terre en mètres
    R = 6371000.0

    # des différences de latitude et de longitude
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calcul de la distance entre les deux points en utilisant la formule de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance / 1000

# Créer une nouvelle colonne 'distance_en_km'
df['distance_en_km'] = df.apply(lambda row: haversine(row['merch_lat'], row['merch_long'], row['lat'], row['long']), axis=1)



In [ ]:
def timeDifference(x):
    x['time_diff'] = x.trans_date_trans_time - x.trans_date_trans_time.shift()
    return x

df= df.groupby(['cc_num']).apply(timeDifference)



In [ ]:
df['time_diff'] = df['time_diff'].dt.seconds

In [ ]:
# Calculer la moyenne des valeurs non nulles de time_diff pour chaque client
avg_time_diff_per_client_train = df.groupby(df['cc_num'])['time_diff'].mean()


# Remplacer les valeurs nulles par la moyenne correspondante de chaque client
df['time_diff'] = df.apply(lambda row: avg_time_diff_per_client_train[row['cc_num']] if pd.isnull(row['time_diff']) else row['time_diff'], axis=1)



In [ ]:
df.isnull().sum().tail(60)

In [ ]:
df

In [ ]:
df_selected = df[(df['cc_num'] == 60416207185	)]


# Affichage des lignes sélectionnées
df_selected

In [ ]:
df.columns

In [ ]:
df.index

In [ ]:
len(df)

In [ ]:
import pandas as pd

# Création d'une fonction pour déterminer la période en fonction de la date de transaction
def determine_period(trans_date):
    day = trans_date.day
    if 1 <= day <= 7:
        return 1
    elif 7 < day < 23:
        return 2
    else:
        return 3

# Ajout de la colonne 'period_first_last' en appliquant la fonction à la colonne 'trans_date_trans_time'
df['period_first_last'] = df['trans_date_trans_time'].apply(determine_period)

# Affichage des premières lignes pour vérification
df[['trans_date_trans_time', 'period_first_last']]


In [ ]:
# Sélection des colonnes 'week_of_year' et 'trans_date_trans_time'
selected_columns = ['year_week']

# Affichage des données
df.loc[:, selected_columns]


In [ ]:
df['year_week'] = pd.to_datetime(df['year_week'])
# Extrait l'année et le numéro de semaine de chaque date
df['week_of_year'] = df['year_week'].dt.isocalendar().week




In [ ]:
df.dtypes

In [ ]:
# Sélection des colonnes 'week_of_year' et 'trans_date_trans_time'
selected_columns = ['week_of_year', 'trans_date_trans_time']

# Affichage desunix_time  données
df.loc[:, selected_columns]


In [ ]:
df

In [ ]:
df.isnull().sum().tail(60)

In [ ]:
df.columns

In [ ]:
'''from sklearn.preprocessing import LabelEncoder

def label_encoding(df):
    # Créer un LabelEncoder
    lb_encoder = LabelEncoder()

    # Parcourir les colonnes du DataFrame
    for col in df.columns:

        if df[col].dtype == 'object'or df[col].dtype == 'category':
            # Appliquer le codage des étiquettes à la colonne
            df[col] = lb_encoder.fit_transform(df[col])

    return df
df= label_encoding(df)'''


In [ ]:
len(df.columns)

In [ ]:
df.columns

In [ ]:
df.dtypes.head(20)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Sauvegarder dans Google Drive
df.to_parquet('/content/drive/MyDrive/PFE/Data sets/df_feature.parquet')



In [ ]:
import pandas as pd
df= pd.read_parquet('/content/drive/MyDrive/PFE/Data sets/df_feature.parquet')


In [ ]:
df

trans_date_trans_time  \
cc_num              trans_date_trans_time                         
60416207185         2020-06-21 13:05:42     2020-06-21 13:05:42   
                    2020-06-21 16:25:36     2020-06-21 16:25:36   
                    2020-06-22 07:58:33     2020-06-22 07:58:33   
                    2020-06-22 15:32:31     2020-06-22 15:32:31   
                    2020-06-23 12:28:54     2020-06-23 12:28:54   
...                                                         ...   
4992346398065154184 2020-12-31 02:10:10     2020-12-31 02:10:10   
                    2020-12-31 05:13:35     2020-12-31 05:13:35   
                    2020-12-31 11:15:44     2020-12-31 11:15:44   
                    2020-12-31 11:17:25     2020-12-31 11:17:25   
                    2020-12-31 13:24:36     2020-12-31 13:24:36   

                                                        cc_num  \
cc_num              trans_date_trans_time                        
60416207185         2020-06-21 13:05:42            60416207185   
                    2020-06-21 16:25:36            60416207185   
                    2020-06-22 07:58:33            60416207185   
                    2020-06-22 15:32:31            60416207185   
                    2020-06-23 12:28:54            60416207185   
...                                                        ...   
4992346398065154184 2020-12-31 02:10:10    4992346398065154184   
                    2020-12-31 05:13:35    4992346398065154184   
                    2020-12-31 11:15:44    4992346398065154184   
                    2020-12-31 11:17:25    4992346398065154184   
                    2020-12-31 13:24:36    4992346398065154184   

                                                                       merchant  \
cc_num              trans_date_trans_time                                         
60416207185         2020-06-21 13:05:42                       fraud_Kutch-Ferry   
                    2020-06-21 16:25:36                   fraud_Halvorson Group   
                    2020-06-22 07:58:33                fraud_Conroy-Cruickshank   
                    2020-06-22 15:32:31                        fraud_Larkin Ltd   
                    2020-06-23 12:28:54                   fraud_Leffler-Goldner   
...                                                                         ...   
4992346398065154184 2020-12-31 02:10:10                 fraud_Parisian and Sons   
                    2020-12-31 05:13:35                 fraud_Bogisich-Homenick   
                    2020-12-31 11:15:44                  fraud_Erdman-Kertzmann   
                    2020-12-31 11:17:25                     fraud_Stracke-Lemke   
                    2020-12-31 13:24:36    fraud_Roberts, Daniel and Macejkovic   

                                                category     amt gender  \
cc_num              trans_date_trans_time                                 
60416207185         2020-06-21 13:05:42             home  124.66      F   
                    2020-06-21 16:25:36         misc_pos   78.52      F   
                    2020-06-22 07:58:33    gas_transport   65.25      F   
                    2020-06-22 15:32:31        kids_pets   87.74      F   
                    2020-06-23 12:28:54    personal_care  148.02      F   
...                                                  ...     ...    ...   
4992346398065154184 2020-12-31 02:10:10    gas_transport   66.11      M   
                    2020-12-31 05:13:35         misc_net    4.58      M   
                    2020-12-31 11:15:44    gas_transport   95.96      M   
                    2020-12-31 11:17:25      grocery_pos  149.48      M   
                    2020-12-31 13:24:36        kids_pets   20.55      M   

                                                      street           city  \
cc_num              trans_date_trans_time                                     
60416207185         2020-06-21 13:05:42     9886 Anita Drive  Fort Washakie   
                  

In [ ]:
df.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'job',
       'unix_time', 'merch_lat', 'merch_long', 'is_fraud', 'age_cust',
       'age_group', 'hour', 'day', 'month', 'hourEnc', 'is_weekend',
       'day_of_week', 'year_week', 'num_transactions', 'avg_amt_1_lastDay',
       'avg_amt_7_lastDay', 'avg_amt_30_lastDay', 'total_amt_1_lastDay',
       'total_amt_7_lastDay', 'total_amt_30_lastDay', 'total_amt_every_day',
       'total_amt_per_30_days', 'total_amt_7_days', 'avg_amt_every_day',
       'avg_amt_per_30_days', 'avg_amt_per_7_days',
       'total_num_transactions_per_day', 'total_num_transactions_per_7_days',
       'total_num_transactions_per_30_days', 'total_num_transactions_lastday',
       'total_num_transactions_last7_days',
       'total_num_transactions_last30_days',
       'total_num_trans_merchant_client_lastday',
       'total_num_trans_merchant_client_last7_days',
       'total_

In [ ]:
'Count_train',y_train.value_counts(),'Count_test',y_test.value_counts(),'Count_val' ,y_val.value_counts()

('Count_train',
 is_fraud
 0    354267
 1      1393
 Name: count, dtype: int64,
 'Count_test',
 is_fraud
 0    110719
 1       425
 Name: count, dtype: int64,
 'Count_val',
 is_fraud
 0    88588
 1      327
 Name: count, dtype: int64)